# Import

In [0]:
import string
from itertools import chain
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from pyspark.ml.feature import *
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

spark = sparknlp.start()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Helper Functions

In [0]:
# HELPER FUNCTIONS
def cleanup_text(text):
    # clean up text data by removing stopwords, punctuation, and lemmatizing words
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words]  
    words = [word for word in words if word not in string.punctuation]   
    words = [lemmatizer.lemmatize(word, pos='v') for word in words] 
    cleaned_text = " ".join(words)
    return cleaned_text

from pyspark.sql.functions import col, size
def add_features(companies):
    # creates new columns based on existing columns to provide additional information
    companies = companies.withColumn('number_of_similar_companies', size(col('similar')))
    companies = companies.withColumn('number_of_locations', size(col('formatted_locations')))
    return companies

def fill_missing_values(companies, features):
    #  fills missing values in the specified columns. For numeric columns, missing values
    # are filled with 0, and for label columns, missing values are filled with 'none'
    for column in features:
        if column in ['number_of_similar_companies', 'employees_in_linkedin', 'followers', 'stability']:
            companies = companies.withColumn(f'{column}', when(companies[column].isNull(), -1).otherwise(companies[column]))
        else:
            companies = companies.withColumn(f'{column}', when(companies[column].isNull(), 'none').otherwise(companies[column]))
    return companies

def feature_to_oneHotEncoder(companies, feature):
    # convert a categorical feature to one-hot encoded vectors
    string_indexer = StringIndexer(inputCol=feature, outputCol=f"{feature}_index")
    one_hot_encoder = OneHotEncoder(inputCol=f"{feature}_index", outputCol=f"{feature}_feature")
    pipeline = Pipeline(stages=[string_indexer, one_hot_encoder])
    companies = pipeline.fit(companies).transform(companies)
    return companies

def category_to_index(companies, feature, dict_index):
    # convert a categorical feature to indexed values
    mapping_expr = create_map([lit(x) for x in chain(*dict_index.items())])
    if feature == 'popularity_category':
        companies = companies.withColumn(f"label", mapping_expr[col(feature)])
    else:
        companies = companies.withColumn(f"{feature}_feature", mapping_expr[col(feature)])
    return companies

def create_word2vec(companies, text_column):
    # create Word2Vec embeddings for a text column
    cleanup_text_udf = udf(cleanup_text)
    punctuation_pattern = r'[^\w\s]'
    companies = companies.withColumn(f'cleaned_{text_column}', F.regexp_replace(col(text_column), punctuation_pattern, ""))
    companies = companies.withColumn(f'cleaned_{text_column}', cleanup_text_udf(col(f'cleaned_{text_column}')))
    # tokenization
    tokenizer = Tokenizer(inputCol=f'cleaned_{text_column}', outputCol=f"{text_column}_tokens")
    companies = tokenizer.transform(companies)
    # learn a Word2Vec model from the tokenized words
    k = 10
    word2vec = Word2Vec(vectorSize=k, minCount=1, inputCol=f"{text_column}_tokens", outputCol=f"{text_column}_feature")
    word2vec_model = word2vec.fit(companies)
    # transform the tokenized words into word vectors
    companies = word2vec_model.transform(companies)
    return companies

def convert_to_label(companies, label):
    # convert a label column to numeric indices
    indexer = StringIndexer(inputCol=label, outputCol=f"label")
    companies = indexer.fit(companies).transform(companies)
    return companies

from pyspark.sql.functions import lit, create_map
def index_to_category(companies, feature, output_col, dict_index):
    # Create a mapping expression to convert indexed values to strings
    dict_index = {v: k for k, v in dict_index.items()}
    reverse_mapping_expr = create_map([lit(v) for v in chain(*dict_index.items())])
    companies = companies.withColumn(output_col, reverse_mapping_expr[col(feature)])
    return companies

from pyspark.sql.functions import col, when
def show_null_precentage(companies, fetures):
    #display the percentage of null values in specified columns
    null_counts = companies.select([sum(col(column).isNull().cast("int")).alias(column) for column in fetures])
    total_rows = companies.count()
    null_percentage = null_counts.select([(col(column) / total_rows * 100).alias(column + "_null_percentage") for column in fetures])
    null_percentage.display()

def embedding_text_features(train_data, test_data, text_features):
    # embedding text columns and converting the resulting array of embeddings into dense vectors
    for feature in text_features:
        # Create a DocumentAssembler for the current feature
        document = DocumentAssembler() \
            .setInputCol(feature) \
            .setOutputCol("document")

        embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_128") \
            .setInputCols(["document"]) \
            .setOutputCol("sentence_embeddings")\
            .setCaseSensitive(True) \
            .setMaxSentenceLength(128)

        pipeline = Pipeline(stages=[document,
                                    embeddings])

        transformed_df = pipeline.fit(train_data).transform(test_data)

        # Join the transformed DataFrame with the original DataFrame using the row_id column
        test_data = test_data.join(transformed_df.select("id", f"sentence_embeddings"), on="id", how="left")
        test_data = test_data.withColumnRenamed("sentence_embeddings", f"{feature}_test")

        # Define a function to convert array to DenseVector
        def array_to_vector(arr):
            return Vectors.dense(arr)
        # Create a UDF from the function
        array_to_vector_udf = udf(array_to_vector, VectorUDT())

        test_data = test_data.withColumn(f"{feature}_feature", flatten(expr(f"transform({feature}_test.embeddings, x -> x)")))
        test_data = test_data.withColumn(f"{feature}_feature", array_to_vector_udf(col(f"{feature}_feature")))

    return test_data 

def create_city_state_mapping(companies):
    companies = companies.withColumn("city", when(companies["headquarters"].isNull(), "other").otherwise(split("headquarters", ",")[0]))
    companies = companies.withColumn("state", when(companies["headquarters"].isNull(), "other").otherwise(split("headquarters", ",")[1]))
    companies = companies.withColumn("state", when(companies["state"].isNull(), "other").otherwise(companies["state"]))

    # Count occurrences of each city and state
    city_counts = companies.groupBy("city").count().alias("city_counts")
    state_counts = companies.groupBy("state").count().alias("state_counts")

    # Filter out cities and states appearing less than 5 times
    city_filtered = city_counts.filter(city_counts["count"] >= 5)
    state_filtered = state_counts.filter(state_counts["count"] >= 2)

    # Create a set of cities and states to replace with "other"
    city_set = set(city_filtered.select("city").rdd.flatMap(lambda x: x).collect())
    state_set = set(state_filtered.select("state").rdd.flatMap(lambda x: x).collect())

    # Replace cities and states not in the set with "other"
    companies = companies.withColumn("filtered_city", when(companies["city"].isin(city_set), companies["city"]).otherwise(lit("other"))) \
                         .withColumn("filtered_state", when(companies["state"].isin(state_set), companies["state"]).otherwise(lit("other")))

    unique_cities = companies.select("filtered_city").distinct().rdd.map(lambda x: x[0]).collect()
    city_dict = {city: index for index, city in enumerate(unique_cities)}

    unique_states = companies.select("filtered_state").distinct().rdd.map(lambda x: x[0]).collect()
    states_dict = {state: index for index, state in enumerate(unique_states)}

    cities = udf(lambda x: city_dict[x])
    states = udf(lambda x: states_dict[x])

    return companies, city_set, state_set, cities, states

def create_city_state_indices(companies, city_set, state_set, cities, states, is_test=False):
    if is_test:
        companies = companies.withColumn("city", when(companies["headquarters"].isNull(), "other").otherwise(split("headquarters", ",")[0]))
        companies = companies.withColumn("state", when(companies["headquarters"].isNull(), "other").otherwise(split("headquarters", ",")[1]))
        companies = companies.withColumn("state", when(companies["state"].isNull(), "other").otherwise(companies["state"]))

        # Replace cities and states not in the set with "other"
        companies = companies.withColumn("filtered_city", when(companies["city"].isin(city_set), companies["city"]).otherwise(lit("other"))) \
                            .withColumn("filtered_state", when(companies["state"].isin(state_set), companies["state"]).otherwise(lit("other")))

    # Create indexes of city and state to one hot vector
    companies = companies.withColumn("city_index", cities(col('filtered_city'))) \
                         .withColumn("state_index", states(col('filtered_state'))) \
                                
    companies = companies.withColumn('city_index', col('city_index').cast('int'))
    companies = companies.withColumn('state_index', col('state_index').cast('int'))
    return companies

def create_city_state_one_hot(old_companies, new_companies):
    encoder = OneHotEncoder(inputCol='city_index', outputCol='city_feature', dropLast=False)
    new_companies = encoder.fit(old_companies.drop('city_feature')).transform(new_companies)

    encoder = OneHotEncoder(inputCol='state_index', outputCol='state_feature', dropLast=False)
    new_companies = encoder.fit(old_companies.drop('state_feature')).transform(new_companies)

    return new_companies

def create_one_hot_from_dict(companies, feature_name, index_dict):
    indices = udf(lambda x: index_dict[x])
    companies = companies.withColumn(f"{feature_name}_index", indices(col(feature_name)))               
    companies = companies.withColumn(f"{feature_name}_index", col(f"{feature_name}_index").cast('int'))
    encoder = OneHotEncoder(inputCol=f"{feature_name}_index", outputCol=f"{feature_name}_feature", dropLast=False)
    companies = encoder.fit(companies).transform(companies)
    return companies


In [0]:
company_size_dict = {'none': 0,
                     '1 employee': 1,
                     '2-10 employees': 2,
                     '11-50 employees': 3,
                     '51-200 employees': 4,
                     '201-500 employees': 5,
                     '501-1,000 employees': 6,
                     '1,001-5,000 employees': 7,
                     '5,001-10,000 employees': 8,
                     '10,001+ employees': 9
                     }

meta_industries_dict = {'Manufacturing': 0,
            'Financial and Investment': 1,
            'Services': 2,
            'Miscellaneous': 3,
            'Healthcare and Medical': 4,
            'Technology': 5,
            'Retail and Consumer Goods': 6,
            'Education and Training': 7,
            'Government and Public Policy': 8,
            'Transportation and Logistics': 9,
            'Real Estate and Construction': 10,
            'Media and Entertainment': 11
            } 

organization_type_dict = {'Privately Held': 0,
                          'Self-Owned': 1,
                          'Nonprofit': 2,
                          'Partnership': 3,
                          'Self-Employed': 4,
                          'Educational': 5,
                          'Public Company': 6,
                          'Government Agency': 7,
                          'none': 8
                        }

label_dict = {"Unpopular": 0,
              "Neutral Popularity": 1,
              "Very Popular": 2
              }


meta_industries_12 = {
    'Furniture and Home Furnishings Manufacturing': 'Manufacturing',
    'Investment Banking': 'Financial and Investment',
    'Architecture and Planning': 'Services',
    'Wholesale': 'Services',
    'Travel Arrangements': 'Services',
    'Ranching': 'Miscellaneous',
    'Hospitals and Health Care': 'Healthcare and Medical',
    'Book and Periodical Publishing': 'Services',
    'Printing Services': 'Services',
    'Professional Training and Coaching': 'Services',
    'Computers and Electronics Manufacturing': 'Manufacturing',
    'Shipbuilding': 'Manufacturing',
    'Public Policy Offices': 'Government and Public Policy',
    'Software Development': 'Technology',
    'Outsourcing and Offshoring Consulting': 'Services',
    'Retail Groceries': 'Retail and Consumer Goods',
    'Education Administration Programs': 'Education and Training',
    'Plastics Manufacturing': 'Manufacturing',
    'Renewable Energy Semiconductor Manufacturing': 'Manufacturing',
    'Computer Networking Products': 'Technology',
    'Events Services': 'Services',
    'Information Services': 'Services',
    'Food and Beverage Services': 'Services',
    'Semiconductor Manufacturing': 'Manufacturing',
    'Business Consulting and Services': 'Services',
    'Insurance': 'Services',
    'Financial Services': 'Services',
    'Wireless Services': 'Services',
    'Computer Hardware Manufacturing': 'Technology',
    'Public Safety': 'Services',
    'Maritime Transportation': 'Transportation and Logistics',
    'Tobacco Manufacturing': 'Manufacturing',
    'Writing and Editing': 'Services',
    'Veterinary Services': 'Services',
    'Staffing and Recruiting': 'Services',
    'Accounting': 'Services',
    'International Affairs': 'Government and Public Policy',
    'Spectator Sports': 'Miscellaneous',
    'Glass, Ceramics and Concrete Manufacturing': 'Manufacturing',
    'Chemical Manufacturing': 'Manufacturing',
    'Mining': 'Miscellaneous',
    'E-Learning Providers': 'Technology',
    'Security and Investigations': 'Services',
    'Translation and Localization': 'Services',
    'Automation Machinery Manufacturing': 'Technology',
    'Computer and Network Security': 'Technology',
    'Political Organizations': 'Government and Public Policy',
    'Environmental Services': 'Government and Public Policy',
    'Oil and Gas': 'Miscellaneous',
    'Real Estate': 'Real Estate and Construction',
    'Think Tanks': 'Government and Public Policy',
    'Executive Offices': 'Miscellaneous',
    'Law Practice': 'Services',
    'Nanotechnology Research': 'Miscellaneous',
    'International Trade and Development': 'Government and Public Policy',
    'Personal Care Product Manufacturing': 'Manufacturing',
    'Philanthropic Fundraising Services': 'Services',
    'Entertainment Providers': 'Media and Entertainment',
    'Market Research': 'Media and Entertainment',
    'Movies, Videos, and Sound': 'Media and Entertainment',
    'Sporting Goods Manufacturing': 'Manufacturing',
    'Graphic Design': 'Services',
    'Technology, Information and Internet': 'Technology',
    'IT Services and IT Consulting': 'Technology',
    'Retail Office Equipment': 'Retail and Consumer Goods',
    'Wholesale Import and Export': 'Services',
    'Capital Markets': 'Financial and Investment',
    'Law Enforcement': 'Services',
    'Freight and Package Transportation': 'Transportation and Logistics',
    'Industrial Machinery Manufacturing': 'Manufacturing',
    'Non-profit Organizations': 'Miscellaneous',
    'Retail Art Supplies': 'Retail and Consumer Goods',
    'Animation and Post-production': 'Media and Entertainment',
    'Transportation, Logistics, Supply Chain and Storage': 'Transportation and Logistics',
    'Aviation and Aerospace Component Manufacturing': 'Transportation and Logistics',
    'Fundraising': 'Financial and Investment',
    'Railroad Equipment Manufacturing': 'Transportation and Logistics',
    'Construction': 'Real Estate and Construction',
    'Investment Management': 'Financial and Investment',
    'Utilities': 'Miscellaneous',
    'Retail Luxury Goods and Jewelry': 'Retail and Consumer Goods',
    'Warehousing and Storage': 'Transportation and Logistics',
    'Media Production': 'Media and Entertainment',
    'Gambling Facilities and Casinos': 'Media and Entertainment',
    'Defense and Space Manufacturing': 'Manufacturing',
    'Facilities Services': 'Services',
    'Government Relations Services': 'Government and Public Policy',
    'Advertising Services': 'Media and Entertainment',
    'Paper and Forest Product Manufacturing': 'Manufacturing',
    'Packaging and Containers Manufacturing': 'Manufacturing',
    'Telecommunications': 'Technology',
    'Medical Equipment Manufacturing': 'Healthcare and Medical',
    'Beverage Manufacturing': 'Manufacturing',
    'Restaurants': 'Retail and Consumer Goods',
    'Leasing Non-residential Real Estate': 'Real Estate and Construction',
    'Newspaper Publishing': 'Media and Entertainment',
    'Armed Forces': 'Miscellaneous',
    'Appliances, Electrical, and Electronics Manufacturing': 'Manufacturing',
    'Hospitality': 'Services',
    'Pharmaceutical Manufacturing': 'Healthcare and Medical',
    'Research Services': 'Services',
    'Retail Apparel and Fashion': 'Retail and Consumer Goods',
    'Photography': 'Media and Entertainment',
    'Wellness and Fitness Services': 'Services',
    'Truck Transportation': 'Transportation and Logistics',
    'Consumer Services': 'Services',
    'Wholesale Building Materials': 'Services',
    'Human Resources Services': 'Services',
    'Airlines and Aviation': 'Transportation and Logistics',
    'Machinery Manufacturing': 'Manufacturing',
    'Individual and Family Services': 'Services',
    'Motor Vehicle Manufacturing': 'Manufacturing',
    'Performing Arts': 'Media and Entertainment',
    'Museums, Historical Sites, and Zoos': 'Media and Entertainment',
    'Broadcast Media Production and Distribution': 'Media and Entertainment',
    'Banking': 'Financial and Investment',
    'Recreational Facilities': 'Miscellaneous',
    'Government Administration': 'Government and Public Policy',
    'Public Relations and Communications Services': 'Media and Entertainment',
    'Fisheries': 'Miscellaneous',
    'Medical Practices': 'Healthcare and Medical',
    'Religious Institutions': 'Miscellaneous',
    'Online Audio and Video Media': 'Media and Entertainment',
    'Artists and Writers': 'Miscellaneous',
    'Biotechnology Research': 'Healthcare and Medical',
    'Legal Services': 'Services',
    'Retail': 'Retail and Consumer Goods',
    'Civil Engineering': 'Services',
    'Libraries': 'Miscellaneous',
    'Alternative Dispute Resolution': 'Miscellaneous',
    'Manufacturing': 'Miscellaneous',
    'Design Services': 'Services',
    'Dairy Product Manufacturing': 'Manufacturing',
    'Higher Education': 'Education and Training',
    'Civic and Social Organizations': 'Miscellaneous',
    'Textile Manufacturing': 'Manufacturing',
    'Venture Capital and Private Equity Principals': 'Financial and Investment',
    'Mental Health Care': 'Healthcare and Medical',
    'Musicians': 'Media and Entertainment',
    'Farming': 'Miscellaneous',
    'Computer Games': 'Media and Entertainment',
    'Strategic Management Services': 'Services',
    'Food and Beverage Manufacturing': 'Manufacturing',
    'Primary and Secondary Education': 'Education and Training',
    'Alternative Medicine': 'Healthcare and Medical',
    'Legislative Offices': 'Services',
    'Administration of Justice': 'Services',
    'Mobile Gaming Apps': 'Media and Entertainment'
}

# Upload new & old companies trend and merge the linkedin companies data

In [0]:
companies = spark.read.parquet('/linkedin/companies')

# upload companies trend
old_companies_trend_df = pd.read_csv("/dbfs/FileStore/shared_uploads/omriitzhaki@campus.technion.ac.il/final_old_companies_trend_preprocess")
new_companies_trend_df = pd.read_csv("/dbfs/FileStore/shared_uploads/omriitzhaki@campus.technion.ac.il/final_new_companies_trend_preprocess")
old_companies = spark.createDataFrame(old_companies_trend_df)
new_companies = spark.createDataFrame(new_companies_trend_df)

# merge with linkedin companies data
old_companies = companies.join(old_companies, 'name', 'inner')
new_companies = companies.join(new_companies, 'name', 'inner')

# Filter companies that contain & in thier name
old_companies = old_companies.filter(~col("search_name").contains("&"))
new_companies = new_companies.filter(~col("search_name").contains("&"))

# filter old & new companies according to our definition
old_companies = old_companies.filter(col('founded') >= '1900').filter(col('founded') < '2020')
new_companies = new_companies.filter(col('founded') >= '2020').filter(col('founded') < '2025')

# add meta_industry column
meta_industry = udf(lambda x: meta_industries_12[x] )

old_companies = old_companies.filter(old_companies.industries.isNotNull())
old_companies = old_companies.withColumn('meta_industry', meta_industry(col('industries')))

new_companies = new_companies.filter(new_companies.industries.isNotNull())
new_companies = new_companies.withColumn('meta_industry', meta_industry(col('industries')))

# Features Preprocess

In [0]:
old_companies = old_companies.withColumn('stability', col('stability').cast('float'))

numeric_features = ['stability', 'number_of_similar_companies']
label = ['popularity']

# add number_of_similar_companies
old_companies = add_features(old_companies)

# fill missing values
old_companies = fill_missing_values(old_companies, ['company_size', 'organization_type', 'stability', 'number_of_similar_companies', 'specialties'])

# stability & number_of_similar_companies features
for feature in ['stability', 'number_of_similar_companies']:
    old_companies = old_companies.withColumn(f'{feature}_feature', col(feature).cast("int"))

# company_size feature
old_companies = category_to_index(old_companies, 'company_size', company_size_dict)

# organization_type feature
old_companies = create_one_hot_from_dict(old_companies, 'organization_type', organization_type_dict)

# meta_industry feature
old_companies = create_one_hot_from_dict(old_companies, 'meta_industry', meta_industries_dict)

# specialties feature
old_companies = embedding_text_features(old_companies, old_companies, ['specialties'])

# popularity label
quantiles = [0.3, 0.7]
categories = ["Unpopular", "Neutral Popularity", "Very Popular"]
popularity_quantiles = old_companies.approxQuantile('popularity', quantiles, 0.01)
old_companies = old_companies.withColumn("popularity_category", 
                                         when(old_companies["popularity"] <= popularity_quantiles[0], lit(categories[0]))
                                         .when((old_companies["popularity"] > popularity_quantiles[0]) & (old_companies["popularity"] <= popularity_quantiles[1]), lit(categories[1]))
                                         .otherwise(lit(categories[2])))

# transform label to indexes
old_companies = category_to_index(old_companies, 'popularity_category', label_dict)

# create feature df after pre process   
features_columns = [col for col in old_companies.columns if 'feature' in col]
features_df = old_companies.select(features_columns + ['label', 'popularity', 'popularity_category', 'id'])

# Assemble numeric features into a vector
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")
features_df = assembler.transform(features_df)

features_df.select('id', 'features', 'popularity_category', 'label').limit(5).display()

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


id,features,popularity_category,label
first-texas-homes,"Map(vectorType -> dense, length -> 152, values -> List(6.0, 10.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.2942283153533936, 0.36948883533477783, 0.4741453230381012, -1.779219627380371, -0.16012081503868103, -0.42470505833625793, -0.5228009223937988, -0.09325098246335983, -1.2865962982177734, -1.3030455112457275, 0.8163958787918091, 0.3656260371208191, 0.1086505651473999, -0.392599880695343, 0.8034310340881348, -0.5854833722114563, -1.7363544702529907, 0.7302411794662476, -1.7386289834976196, 1.4275147914886475, 0.37002477049827576, -0.4076819121837616, 0.738832950592041, 0.5336618423461914, 2.2741668224334717, -0.8179621696472168, 0.7329664826393127, -0.7968835234642029, 0.5774044990539551, -2.2542810440063477, -1.0446972846984863, -1.4977399110794067, -1.3070158958435059, 1.3714115619659424, -0.31685546040534973, -0.19211356341838837, 0.058205995708703995, 0.7681118845939636, -1.2787091732025146, -1.697519302368164, -0.45648184418678284, -0.26102563738822937, 0.24745506048202515, -0.9577698707580566, -0.0667509064078331, -1.1633553504943848, -0.2673433721065521, -0.49115926027297974, 0.5277093052864075, 0.09757150709629059, -0.6518255472183228, 0.7394610643386841, 0.6812499761581421, -0.29619669914245605, -1.3591455221176147, -0.2271805852651596, 1.1283392906188965, 0.24180689454078674, 0.13638624548912048, 1.7329740524291992, -0.4629392921924591, -0.4326169490814209, -0.6684926748275757, -0.02385726571083069, -1.4461091756820679, -1.3901067972183228, 0.17803633213043213, 1.1983410120010376, 1.047681450843811, 0.7665719389915466, -0.23092703521251678, -0.30814558267593384, -0.710768461227417, 0.5776767134666443, -1.6258842945098877, 1.0848305225372314, 0.6421483159065247, -0.013245359063148499, 2.7783584594726562, -0.3264310359954834, -0.6485590934753418, 0.01564553566277027, -0.20050643384456635, -0.33974412083625793, 0.13228704035282135, -0.992781937122345, -1.9872535467147827, 0.4659959673881531, 1.3718551397323608, -1.688912272453308, 1.0149595737457275, 1.3278608322143555, 0.09444236755371094, 1.7074025869369507, 0.42837104201316833, 1.155930519104004, 0.887734591960907, -0.3426534831523895, -0.6012884974479675, 1.3989413976669312, -1.0866906642913818, 0.5543171763420105, 0.30536088347435, 0.3028407096862793, 1.0616294145584106, -1.0390998125076294, -2.4529764652252197, 1.0342470407485962, -0.7142607569694519, -0.9097476005554199, 1.1186363697052002, -0.006872878409922123, -0.3037011921405792, 0.611248254776001, 1.3175405263900757, 0.7859256863594055, -0.12060131877660751, 0.7981509566307068, 0.3981960117816925, 0.41766536235809326, 0.6601080298423767, 0.08885040134191513, 1.4190430641174316, -1.5245566368103027, -1.2806589603424072, -0.8480520844459534, -1.4083153009414673, 1.6311087608337402))",Neutral Popularity,1
the-revolution-hotel,"Map(vectorType -> dense, length -> 152, values -> List(17.0, 10.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.6838653683662415, -0.027924669906497, -0.22845515608787537, -1.8258554935455322, 1.3247026205062866, -0.0275909211486578, 0.08124829083681107, 1.4760819673538208, -1.2548776865005493, -0.5722838640213013, 1.4547723531723022, 0.35236525535583496, 1.299685001373291, -0.4803466200828552, 1.4261395931243896, -0.5757228136062622, 0.3611500859260559, -0.9588012099266052, -0.7471068501472473, 1.7423712015151978, 0.03180694580078125, 1.3310972452163696, 1.5403541326522827, 0.9598807692527771, 1.1077752113342285, -0.5975169539451599, 0.5360548496246338, 0.37211617827415466, 0.09181081503629684, -0.3965241014957428, -0.8349302411079407, -1.803664207458496, -0.8927216529846191, -0.23821167647838593, 0.3768949806690216, -0.03050830587744713, -1.1166585683822632, -0.9257135391235352, -2.385453462600708, -1.6240590810775757, 0.3678542673587799, 0.7329257130622864, 1.3856385946273804, -0.942237079

# Train the model

In [0]:
from pyspark.ml.classification import *
from pyspark.mllib.evaluation import MulticlassMetrics

data = features_df.select("features", "label")
# Split data into training and test sets
(train_data, test_data) = data.randomSplit([0.8, 0.2], seed=87)

model = LogisticRegression(labelCol="label", featuresCol="features")
model_name = 'Logistic Regression'
print(f"Training {model_name}...")
trained_model = model.fit(train_data)

predictions = trained_model.transform(test_data)
    
# Calculate accuracy
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
accuracy = accuracy_evaluator.evaluate(predictions)
print(f"Accuracy for {model_name}: {accuracy}")

# Calculate F1
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol='prediction', metricName='weightedFMeasure')
f1 = f1_evaluator.evaluate(predictions)
print(f"F1 for {model_name}: {f1}")

# Calculate confusion matrix
predictionAndLabels = predictions.select("prediction", "label").rdd.map(lambda x: (float(x[0]), float(x[1])))
metrics = MulticlassMetrics(predictionAndLabels)
confusion_matrix = metrics.confusionMatrix().toArray()
print(f"Confusion matrix for {model_name}:\n{confusion_matrix}")

Training Logistic Regression...
Accuracy for Logistic Regression: 0.49886415265788275
F1 for Logistic Regression: 0.49406166064867985
/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
Confusion matrix for Logistic Regression:
[[381. 180. 115.]
 [201. 465. 157.]
 [145. 305. 252.]]


# New_companies features preprocess

In [0]:
predicted_stability_df = spark.read.csv("/dbfs/FileStore/shared_uploads/omriitzhaki@campus.technion.ac.il/predicted_stability_df", header=True)
predicted_stability_df = predicted_stability_df.withColumnRenamed("id", "new_company_id")

new_companies = new_companies.join(predicted_stability_df, new_companies['id'] == predicted_stability_df['new_company_id'], 'inner')

In [0]:
# add number_of_similar_companies
new_companies = add_features(new_companies)

# fill missing values
new_companies = fill_missing_values(new_companies, ['company_size', 'organization_type', 'predicted_stability', 'number_of_similar_companies'])

# predicted_stability & number_of_similar_companies features
for feature in ['predicted_stability', 'number_of_similar_companies']:
    new_companies = new_companies.withColumn(f'{feature}_feature', col(feature).cast("int"))

# company_size feature
new_companies = category_to_index(new_companies, 'company_size', company_size_dict)

# organization_type feature
new_companies = create_one_hot_from_dict(new_companies, 'organization_type', organization_type_dict)

# meta_industry features
new_companies = create_one_hot_from_dict(new_companies, 'meta_industry', meta_industries_dict)

# specialties features
new_companies = embedding_text_features(old_companies, new_companies, ['specialties'])

# create feature df after pre process   
features_columns = [col for col in new_companies.columns if 'feature' in col]
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")
new_companies = assembler.transform(new_companies)

new_companies.select('id', 'features').limit(5).display()

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


id,features
28-research-inc,"Map(vectorType -> dense, length -> 152, values -> List(7.0, 10.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.064393162727356, 0.7143968343734741, 0.5241509079933167, -0.6406130194664001, -0.4820651710033417, -0.31723058223724365, -0.3947852551937103, -0.3411380648612976, -1.3921515941619873, -1.5019888877868652, 0.2138400673866272, 0.9520403742790222, 0.5377432703971863, -0.5264187455177307, 0.9428620934486389, -0.2981928884983063, -1.5826939344406128, 0.9418190717697144, -2.1611194610595703, 1.4397075176239014, 0.35751813650131226, -0.5008982419967651, 0.7560628652572632, 0.9848535656929016, 2.29754638671875, -0.9613281488418579, 0.6152336597442627, -1.2238579988479614, 0.2039899230003357, -2.390124559402466, -1.2434908151626587, -1.7995260953903198, -1.3016014099121094, 1.5023844242095947, -0.2026672065258026, -0.45338794589042664, -0.5167436003684998, 0.962124764919281, -1.303858757019043, -1.257920503616333, -0.4217424690723419, 0.17379410564899445, -0.8672763705253601, -0.9983976483345032, 0.07962798327207565, -1.113377332687378, -0.49949967861175537, -0.337821364402771, 0.12731625139713287, 0.3744203448295593, -0.32887905836105347, 0.7664983868598938, 0.8250643014907837, -0.4037265479564667, -1.6031090021133423, -0.039046160876750946, 0.9913279414176941, 0.28838279843330383, 0.5828094482421875, 1.5232658386230469, -0.4500977694988251, -0.3883506953716278, -0.7256516218185425, -0.023224465548992157, -1.3720935583114624, -1.3057318925857544, 0.2295370101928711, 1.0049102306365967, 0.16994470357894897, 0.9507079124450684, -0.4002135396003723, -0.4196605682373047, -0.5182173848152161, 0.4381556808948517, -1.7948323488235474, 1.0571529865264893, 0.4924606382846832, 0.017973685637116432, 2.4169158935546875, -0.9335283637046814, -0.4382433295249939, -0.515625, -0.20875521004199982, -0.22134999930858612, 0.7794089317321777, -0.9638808965682983, -1.8447508811950684, 0.7349623441696167, 0.6650119423866272, -1.1374192237854004, 1.399522066116333, 1.3147553205490112, 0.11778252571821213, 1.6046675443649292, 0.14102590084075928, 1.1515684127807617, 0.7832249402999878, -0.15692463517189026, -0.5074137449264526, 1.4322956800460815, -0.6361590027809143, 0.9738180637359619, 0.29875659942626953, 0.09705051779747009, 1.2342935800552368, -1.0303230285644531, -2.724984645843506, 1.3756096363067627, -0.7404832243919373, -0.18294018507003784, 0.8243940472602844, 0.05158580467104912, -0.08523862063884735, 0.5111402869224548, 0.7751591205596924, 0.46444055438041687, -0.23535938560962677, 0.9081387519836426, 0.35983845591545105, 0.42357540130615234, 0.9399452805519104, 0.3212769627571106, 1.2495023012161255, -1.3127309083938599, -1.6020816564559937, -0.5511237382888794, -1.5033149719238281, 1.3086556196212769))"
3bulldogs,"Map(vectorType -> dense, length -> 152, values -> List(6.0, 7.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0880430936813354, 0.9748651385307312, -1.6383212804794312, -2.8524060249328613, -0.7699288129806519, 0.34203633666038513, -0.11622174829244614, 0.33084622025489807, -1.3546890020370483, 1.4995778799057007, 1.3680124282836914, -0.24632102251052856, -2.2074813842773438, -0.045035574585199356, 1.1248810291290283, -1.528263807296753, -0.4127964377403259, 0.14344491064548492, -2.2425012588500977, 1.319778561592102, 0.03117082081735134, -0.22839972376823425, 2.3721423149108887, 1.7022351026535034, 0.8859752416610718, 0.09486593306064606, 0.5028294324874878, 0.39916419982910156, 0.19746647775173187, 0.7801249027252197, -1.1860527992248535, -1.2983670234680176, -0.5965096950531006, -0.03278372064232826, 1.1925708055496216, -0.7110944986343384, 0.6308732032775879, -1.0647984743118286, -1.4240450859069824, -1.4944943189620972, 1.2783164978027344, -0.5234233736991882, 1.5809653997421265, -0.5956752300262451, -0.0438213013112545, -1.3817330598831177, -0.35168176889419

# New_companies Predict Popularity 

In [0]:
preds = trained_model.transform(new_companies)
preds = index_to_category(preds, 'prediction', 'predicted_popularity', label_dict)
preds.select('id', 'predicted_popularity', 'prediction').limit(10).display()

id,predicted_popularity,prediction
redwood-services-llc,Neutral Popularity,1.0
bunnystudio,Very Popular,2.0
splitappstore,Very Popular,2.0
test-banks,Very Popular,2.0
workshoplocal,Neutral Popularity,1.0
eucalyptushealthcare,Very Popular,2.0
shopclairvaux,Very Popular,2.0
theculturexyz,Very Popular,2.0
in-the-worx,Neutral Popularity,1.0
vita-health,Neutral Popularity,1.0


In [0]:
preds.select('id', 'predicted_popularity').write.csv("/dbfs/FileStore/shared_uploads/omriitzhaki@campus.technion.ac.il/predicted_popularity_df", header=True)